# pandas

> Stata-to-pandas utilities, used in `nbstata.browse`
- order: 6

In [ ]:
#| default_exp pandas
%load_ext autoreload
%autoreload 2

[autoreload of fastcore.basics failed: Traceback (most recent call last):
  File "c:\Users\tjhuegerich\AppData\Local\anaconda3\envs\nbstata_dev\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\tjhuegerich\AppData\Local\anaconda3\envs\nbstata_dev\Lib\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "c:\Users\tjhuegerich\AppData\Local\anaconda3\envs\nbstata_dev\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\Users\tjhuegerich\AppData\Local\anaconda3\envs\nbstata_dev\Lib\site-packages\IPython\extensions\autoreload.py", line 365, in update_class
    update_instances(old, new)
  File "c:\Users\tjhuegerich\AppData\Local\anaconda3\envs\nbstata_dev\Lib\site-packages\IPython\extensions\autoreload.py", line 323, in update_instances
    object.__setattr__(ref, "__class__", new)
TypeErr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from nbstata.stata import stata_formatted
from nbstata.stata_more import IndexVar

## Better stata-to-pandas

Improves on [pystata.stata.pdataframe_from_data](https://www.stata.com/python/pystata18/stata.html#pystata.stata.pdataframe_from_data) (and 
[pystata.stata.pdataframe_from_frame](https://www.stata.com/python/pystata18/stata.html#pystata.stata.pdataframe_from_frame)) by making the pandas DataFrame index correspond to Stata observation numbers and also providing an option to output numeric values as strings using their Stata formats.

In [ ]:
#| export
def _better_dataframe(hdl, var, obs, selectvar, valuelabel, missingval):
    import pandas as pd
    with IndexVar() as idx_var:
        data = hdl.getAsDict(var, obs, selectvar, valuelabel, missingval)
        if not data:
            return pd.DataFrame()
    
        if idx_var in data:
            idx = data.pop(idx_var)
        else:
            temp_var = [idx_var, selectvar] if selectvar else idx_var
            idx = hdl.getAsDict(temp_var, obs, selectvar, valuelabel, missingval).pop(idx_var)
        idx = pd.array(idx, dtype='int64')

        return pd.DataFrame(data=data, index=idx)

#| hide

Reference Stata docs: [getAsDict example](https://blog.stata.com/2020/11/05/stata-python-integration-part-8-using-the-stata-function-interface-to-copy-data-from-stata-to-python/)

In [ ]:
#| hide
#| eval: False
from nbstata.config import launch_stata
from nbstata.stata_more import run_sfi

In [ ]:
#| hide
#| eval: False
launch_stata(splash=False)

In [ ]:
#| hide
#| eval: False
import sfi

In [ ]:
#| hide
from fastcore.test import test_eq, test_fail
from numpy import nan

In [ ]:
#| hide
#| eval: False
run_sfi("""\
sysuse uslifeexp2, clear
gen sel_var = year==1920""")

(US life expectancy, 1900–1940)


In [ ]:
#| hide
#| eval: False
_better_dataframe(sfi.Data, 'le', None, 'sel_var', True, nan)

,le
21,54.099998


In [ ]:
#| hide
#| eval: False
_better_dataframe(sfi.Data, None, range(0, 10), 'sel_var', True, nan)

""


In [ ]:
#| hide
#| eval: False
_better_dataframe(sfi.Data, None, range(-31, -1), 'sel_var', True, nan)

,year,le,sel_var
21,1920,54.099998,1.0


In [ ]:
#| hide
#| eval: False
run_sfi("drop sel_var")

In [ ]:
#| export
def _simple_dataframe_from_stata(stfr, var, valuelabel, missingval):
    from pystata import stata
    if stfr is None:
        df = stata.pdataframe_from_data(var=var, valuelabel=valuelabel, missingval=missingval)
    else:
        df = stata.pdataframe_from_frame(stfr, var=var, valuelabel=valuelabel, missingval=missingval)
    df.index += 1
    return df

In [ ]:
#| hide
#| eval: False
from pystata.stata import _DefaultMissing

In [ ]:
#| hide
#| eval: False
run_sfi("""\
sysuse lifeexp, clear
keep in 1/5""")
_simple_dataframe_from_stata(None, None, False, _DefaultMissing())

(Life expectancy, 1998)
(63 observations deleted)


,region,country,popgrowth,lexp,gnppc,safewater
1,1,Albania,1.2,72,810.0,7.600000e+01
2,1,Armenia,1.1,74,460.0,8.988466e+307
3,1,Austria,0.4,79,26830.0,8.988466e+307
4,1,Azerbaijan,1.4,71,480.0,8.988466e+307
5,1,Belarus,0.3,68,2180.0,8.988466e+307


In [ ]:
#| export
def better_dataframe_from_stata(stfr, var, obs, selectvar, valuelabel, missingval, sformat):
    from numpy import nan
    import pandas as pd
    import sfi
    hdl = sfi.Data if stfr is None else sfi.Frame.connect(stfr)
    custom_index_not_needed = obs is None and not selectvar
    if custom_index_not_needed:
        df = _simple_dataframe_from_stata(stfr, var, valuelabel, missingval)
    else:
        if hdl.getObsTotal() <= 0:
            return pd.DataFrame()
        df = _better_dataframe(hdl, var, obs, selectvar, valuelabel, missingval)
    if sformat:
        for v in list(df.columns):
            if hdl.isVarTypeString(v) or (valuelabel and missingval==nan
                                          and not pd.api.types.is_numeric_dtype(df[v])):
                continue
            v_format = hdl.getVarFormat(v)
            if missingval != nan and not pd.api.types.is_numeric_dtype(df[v]):
                def format_value(x):
                    return stata_formatted(x, v_format).lstrip() if type(x)!=str else x
            else:
                def format_value(x):
                    return stata_formatted(x, v_format).lstrip()
            df[v] = df[v].apply(format_value)
    return df

In [ ]:
#| export
def better_pdataframe_from_data(var=None, obs=None, selectvar=None, valuelabel=False, missingval=None, sformat=False):
    from numpy import nan
    if missingval is None:
        missingval = nan
    return better_dataframe_from_stata(None, var, obs, selectvar, valuelabel, missingval, sformat)

In [ ]:
#| export
def better_pdataframe_from_frame(stfr, var=None, obs=None, selectvar=None, valuelabel=False, missingval=None, sformat=False):
    from numpy import nan
    if missingval is None:
        missingval = nan
    return better_dataframe_from_stata(stfr, var, obs, selectvar, valuelabel, missingval, sformat)

In [ ]:
#| eval: False
run_sfi("""\
sysuse uslifeexp2, clear
replace le = . if _n==5
replace year = 2022 if year==1900
format year %-ty
gen str_var = "test string" if _n!=5
gen date = dofy(year)
format date %td
gen double date_tc = cofd(date)
format date_tc %tc
gen double date_tC = Cofd(date)
format date_tC %tC""")
better_pdataframe_from_data().head()

(US life expectancy, 1900–1940)
(1 real change made, 1 to missing)
(1 real change made)
(1 missing value generated)


,year,le,str_var,date,date_tc,date_tC
1,2022,47.299999,test string,22646.0,1.956614e+12,1.956614e+12
2,1901,49.099998,test string,-21549.0,-1.861834e+12,-1.861834e+12
3,1902,51.500000,test string,-21184.0,-1.830298e+12,-1.830298e+12
4,1903,50.500000,test string,-20819.0,-1.798762e+12,-1.798762e+12
5,1904,NaN,,-20454.0,-1.767226e+12,-1.767226e+12


In [ ]:
#| eval: False
better_pdataframe_from_data(sformat=True).head()

,year,le,str_var,date,date_tc,date_tC
1,2022,47.3,test string,01jan2022,01jan2022 00:00:00,01jan2022 00:00:00
2,1901,49.1,test string,01jan1901,01jan1901 00:00:00,01jan1901 00:00:00
3,1902,51.5,test string,01jan1902,01jan1902 00:00:00,01jan1902 00:00:00
4,1903,50.5,test string,01jan1903,01jan1903 00:00:00,01jan1903 00:00:00
5,1904,.,,01jan1904,01jan1904 00:00:00,01jan1904 00:00:00


In [ ]:
#| hide
#| eval: False
better_pdataframe_from_data(valuelabel=True, sformat=True, missingval=".").head()

,year,le,str_var,date,date_tc,date_tC
1,2022,47.3,test string,01jan2022,01jan2022 00:00:00,01jan2022 00:00:00
2,1901,49.1,test string,01jan1901,01jan1901 00:00:00,01jan1901 00:00:00
3,1902,51.5,test string,01jan1902,01jan1902 00:00:00,01jan1902 00:00:00
4,1903,50.5,test string,01jan1903,01jan1903 00:00:00,01jan1903 00:00:00
5,1904,.,,01jan1904,01jan1904 00:00:00,01jan1904 00:00:00


In [ ]:
#| hide
#| eval: False
better_pdataframe_from_data(sformat=True, missingval=".").head()

,year,le,str_var,date,date_tc,date_tC
1,2022,47.3,test string,01jan2022,01jan2022 00:00:00,01jan2022 00:00:00
2,1901,49.1,test string,01jan1901,01jan1901 00:00:00,01jan1901 00:00:00
3,1902,51.5,test string,01jan1902,01jan1902 00:00:00,01jan1902 00:00:00
4,1903,50.5,test string,01jan1903,01jan1903 00:00:00,01jan1903 00:00:00
5,1904,.,,01jan1904,01jan1904 00:00:00,01jan1904 00:00:00


In [ ]:
#| hide
#| eval: False
better_pdataframe_from_data(obs=range(1, 3)).head()

,year,le,str_var,date,date_tc,date_tC
2,1901,49.099998,test string,-21549.0,-1.861834e+12,-1.861834e+12
3,1902,51.500000,test string,-21184.0,-1.830298e+12,-1.830298e+12


In [ ]:
#| hide
#| eval: False
test_fail(better_pdataframe_from_data, kwargs=dict(obs=range(1000)))

In [ ]:
#| hide
#| eval: False
better_pdataframe_from_data(missingval='.').head()

,year,le,str_var,date,date_tc,date_tC
1,2022,47.299999,test string,22646.0,1.956614e+12,1.956614e+12
2,1901,49.099998,test string,-21549.0,-1.861834e+12,-1.861834e+12
3,1902,51.5,test string,-21184.0,-1.830298e+12,-1.830298e+12
4,1903,50.5,test string,-20819.0,-1.798762e+12,-1.798762e+12
5,1904,.,,-20454.0,-1.767226e+12,-1.767226e+12


In [ ]:
#| hide
#| eval: False
better_pdataframe_from_data(var=['year', 'date_tc', 'date_tC']).head()

,year,date_tc,date_tC
1,2022,1.956614e+12,1.956614e+12
2,1901,-1.861834e+12,-1.861834e+12
3,1902,-1.830298e+12,-1.830298e+12
4,1903,-1.798762e+12,-1.798762e+12
5,1904,-1.767226e+12,-1.767226e+12


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

AttributeError: module 'nbdev' has no attribute 'export'